# Imports

In [1]:
import pandas as pd
import numpy as np

# Data

In [2]:
bestand = '.\data\dag8.txt'
bestand_test = '.\data\\voorbeeld8.txt'

with open(bestand, 'r') as file:
    data = (file
            .read()
            .splitlines()
            )
    
with open(bestand_test, 'r') as file:
    test_data = (file
            .read()
            .splitlines()
            )


# Dag 8.1

In [3]:
def make_df(data):
    df = (
        pd.DataFrame(data)[0]
        .str.split('', expand=True)
        .replace('', np.nan)
        .dropna(axis=1)
        .reset_index(drop=True)
        )
    df.columns = range(df.columns.size)
    return df

#test
assert make_df(['...', '...', '...']).equals(pd.DataFrame('.', index=np.arange(3), columns=np.arange(3)) )

In [4]:
def find_unique_antennas_in_list_item(list_items):
    return list(set(list_items))


assert find_unique_antennas_in_list_item('.....0') == ['0', '.']

In [5]:
def find_unique_sorts_of_antennas(data):
    unique_list_items = set(data)
    unique_sorts_of_antennas = []
    for list_items in unique_list_items:
        unique_sorts_of_antennas_in_this_list = find_unique_antennas_in_list_item(list_items)
        unique_sorts_of_antennas = find_unique_antennas_in_list_item(unique_sorts_of_antennas_in_this_list+unique_sorts_of_antennas)
    
    return unique_sorts_of_antennas

find_unique_sorts_of_antennas(['jdie', 'jdi9'])
# #test
assert find_unique_sorts_of_antennas(test_data).sort() == ['0', 'A', '.'].sort() 
assert find_unique_sorts_of_antennas(['jdie', 'jdi9']).sort()  == ['e', 'i', 'd', '9', 'j'].sort() 

In [6]:
def find_all_antenna_coordinates(df, antennatype):
    coordinates_tuple = np.nonzero(df.isin([antennatype]))
    list_antennas = []
    for i in range(len(coordinates_tuple[0])):
        list_antennas += [(coordinates_tuple[0][i], coordinates_tuple[1][i])]
    return list_antennas    
    

#test
df_test = pd.DataFrame('.', index=np.arange(9), columns=np.arange(9)) 
df_test.iloc[1,1] = 'A'
df_test.iloc[4,4] = 'A'
df_test.iloc[6,2] = 'A'
df_test.iloc[7,2] = 'Z'
assert find_all_antenna_coordinates(df_test, 'A') == [(1,1), (4,4), (6,2)]
assert find_all_antenna_coordinates(df_test, 'Z') == [(7,2)]

In [13]:
def coordinate_valid(row_index, col_index, size_df):
    if (row_index < 0) | (col_index < 0) | (row_index >= size_df) | (col_index >= size_df):
        return False
    else:
        return True   

In [15]:
def find_coordinates_of_antinodes(combination_antenna, size_df):
    coordinates_antinodes = []
    coordinate_antenna_1 = combination_antenna[0]
    coordinate_antenna_2 = combination_antenna[1]
    row_index_direction = abs(coordinate_antenna_1[0] - coordinate_antenna_2[0])
    col_index_direction = abs(coordinate_antenna_1[1] - coordinate_antenna_2[1])
    
    if (coordinate_antenna_1[0] >= coordinate_antenna_2[0]) & (coordinate_antenna_1[1] >= coordinate_antenna_2[1]):
        row_coordinate_antinode_1 = coordinate_antenna_1[0] + row_index_direction
        col_coordinate_antinode_1 = coordinate_antenna_1[1] + col_index_direction
        
        row_coordinate_antinode_2 = coordinate_antenna_2[0] - row_index_direction
        col_coordinate_antinode_2 = coordinate_antenna_2[1] - col_index_direction

    elif (coordinate_antenna_1[0] >= coordinate_antenna_2[0]) & (coordinate_antenna_1[1] < coordinate_antenna_2[1]):
        row_coordinate_antinode_1 = coordinate_antenna_1[0] + row_index_direction
        col_coordinate_antinode_1 = coordinate_antenna_1[1] - col_index_direction
        
        row_coordinate_antinode_2 = coordinate_antenna_2[0] - row_index_direction
        col_coordinate_antinode_2 = coordinate_antenna_2[1] + col_index_direction

    elif (coordinate_antenna_1[0] < coordinate_antenna_2[0]) & (coordinate_antenna_1[1] >= coordinate_antenna_2[1]):
        row_coordinate_antinode_1 = coordinate_antenna_1[0] - row_index_direction
        col_coordinate_antinode_1 = coordinate_antenna_1[1] + col_index_direction
        
        row_coordinate_antinode_2 = coordinate_antenna_2[0] + row_index_direction
        col_coordinate_antinode_2 = coordinate_antenna_2[1] - col_index_direction
    
    elif (coordinate_antenna_1[0] < coordinate_antenna_2[0]) & (coordinate_antenna_1[1] < coordinate_antenna_2[1]):
        row_coordinate_antinode_1 = coordinate_antenna_1[0] - row_index_direction
        col_coordinate_antinode_1 = coordinate_antenna_1[1] - col_index_direction
        
        row_coordinate_antinode_2 = coordinate_antenna_2[0] + row_index_direction
        col_coordinate_antinode_2 = coordinate_antenna_2[1] + col_index_direction
    
    
    if coordinate_valid(row_coordinate_antinode_1,col_coordinate_antinode_1 , size_df):
        coordinates_antinodes += [(row_coordinate_antinode_1, col_coordinate_antinode_1)]

    if coordinate_valid(row_coordinate_antinode_2 ,col_coordinate_antinode_2, size_df ):
        coordinates_antinodes += [(row_coordinate_antinode_2, col_coordinate_antinode_2)]
    
    return coordinates_antinodes

assert find_coordinates_of_antinodes([(1,1), (4,4)], 9) == [(7,7)]
assert find_coordinates_of_antinodes([(6,2), (4,4)], 9) == [(8,0), (2,6)]
assert find_coordinates_of_antinodes([(2,1), (4,1)], 9) == [(0,1), (6,1)]
assert find_coordinates_of_antinodes([(2,6), (4,4)], 9) == [(0,8), (6,2)]
assert find_coordinates_of_antinodes([(2,2), (4,4)], 9) == [(0,0), (6,6)]
assert find_coordinates_of_antinodes([(6,6), (4,4)], 9) == [(8,8), (2,2)]

In [44]:
def add_new_antinodes_for_this_antennatype(df, antennatype, antinodes, part=1):
    new_antinodes_add = antinodes.copy()
    list_of_all_antenna_coordinates = find_all_antenna_coordinates(df, antennatype)
    number_of_antinodes = len(list_of_all_antenna_coordinates)
    for i in range(number_of_antinodes): 
        for j in range(i+1, number_of_antinodes):
            combination_antenna = [list_of_all_antenna_coordinates[i], list_of_all_antenna_coordinates[j]]
            if part == 1:
                coordinates_of_antinodes = find_coordinates_of_antinodes(combination_antenna, df.shape[0])
            else:
                coordinates_of_antinodes = find_coordinates_of_antinodes_part2(combination_antenna, df.shape[0])
                new_antinodes_add.iloc[list_of_all_antenna_coordinates[i][0]][list_of_all_antenna_coordinates[i][1]] = 'X'
                new_antinodes_add.iloc[list_of_all_antenna_coordinates[j][0]][list_of_all_antenna_coordinates[j][1]] = 'X'
            for coordinates in coordinates_of_antinodes:
                new_antinodes_add.iloc[coordinates[0]][coordinates[1]] = 'X'
    return new_antinodes_add
    

#test
df_test = pd.DataFrame('.', index=np.arange(9), columns=np.arange(9)) 
df_test.iloc[1,1] = 'B'
df_test.iloc[4,4] = 'B'
df_test.iloc[6,2] = 'B'
df_antinodes_test = pd.DataFrame('.', index=np.arange(9), columns=np.arange(9)) 
df_antinodes_test_start = df_antinodes_test.copy() 
df_antinodes_test.iloc[7,7] = 'X'
df_antinodes_test.iloc[8,0] = 'X'
df_antinodes_test.iloc[2,6] = 'X'
assert add_new_antinodes_for_this_antennatype( df_test, 'B', df_antinodes_test_start).equals(df_antinodes_test)
#test 2
df_test = pd.DataFrame('.', index=np.arange(12), columns=np.arange(12)) 
df_test.iloc[5,6] = 'H'
df_test.iloc[8,8] = 'H'
df_test.iloc[9,9] = 'H'
df_antinodes_test = pd.DataFrame('.', index=np.arange(12), columns=np.arange(12)) 
df_antinodes_test_start = df_antinodes_test.copy() 
df_antinodes_test.iloc[2,4] = 'X'
df_antinodes_test.iloc[11,10] = 'X'
df_antinodes_test.iloc[1,3] = 'X'
df_antinodes_test.iloc[7,7] = 'X'
df_antinodes_test.iloc[10,10] = 'X'
assert add_new_antinodes_for_this_antennatype( df_test, 'H', df_antinodes_test_start).equals(df_antinodes_test)


In [45]:
def count_all_antinodes(data):
    df =  make_df(data)
    sorts_of_antennas = find_unique_sorts_of_antennas(data)
    sorts_of_antennas.remove('.')
    antinodes = pd.DataFrame(columns=df.columns, index=df.index)
    for antennatype in sorts_of_antennas:
        antinodes = add_new_antinodes_for_this_antennatype(df, antennatype, antinodes)
    number_of_antinodes = (antinodes == 'X').sum().sum()
    return number_of_antinodes

#test
assert count_all_antinodes(test_data) == 14

In [46]:
count_all_antinodes(data)

320

# Dag 8.2

In [ ]:
def find_coordinates_of_antinodes_part2(combination_antenna, size_df):
    coordinates_antinodes = []
    coordinate_antenna_1 = combination_antenna[0]
    coordinate_antenna_2 = combination_antenna[1]
    row_index_direction = abs(coordinate_antenna_1[0] - coordinate_antenna_2[0])
    col_index_direction = abs(coordinate_antenna_1[1] - coordinate_antenna_2[1])
    
    if (coordinate_antenna_1[0] >= coordinate_antenna_2[0]) & (coordinate_antenna_1[1] >= coordinate_antenna_2[1]):
                
        row_coordinate_antinode_1 = coordinate_antenna_1[0] + row_index_direction
        col_coordinate_antinode_1 = coordinate_antenna_1[1] + col_index_direction
        
        row_coordinate_antinode_2 = coordinate_antenna_2[0] - row_index_direction
        col_coordinate_antinode_2 = coordinate_antenna_2[1] - col_index_direction

        while coordinate_valid(row_coordinate_antinode_1, col_coordinate_antinode_1, size_df):
            coordinates_antinodes += [(row_coordinate_antinode_1, col_coordinate_antinode_1)]
            row_coordinate_antinode_1 = row_coordinate_antinode_1 + row_index_direction
            col_coordinate_antinode_1 = col_coordinate_antinode_1 + col_index_direction
        
        while coordinate_valid(row_coordinate_antinode_2, col_coordinate_antinode_2, size_df):
            coordinates_antinodes += [(row_coordinate_antinode_2, col_coordinate_antinode_2)]  
            row_coordinate_antinode_2 = row_coordinate_antinode_2 - row_index_direction
            col_coordinate_antinode_2 = col_coordinate_antinode_2 - col_index_direction
  

    elif (coordinate_antenna_1[0] >= coordinate_antenna_2[0]) & (coordinate_antenna_1[1] < coordinate_antenna_2[1]):
        row_coordinate_antinode_1 = coordinate_antenna_1[0] + row_index_direction
        col_coordinate_antinode_1 = coordinate_antenna_1[1] - col_index_direction
        
        row_coordinate_antinode_2 = coordinate_antenna_2[0] - row_index_direction
        col_coordinate_antinode_2 = coordinate_antenna_2[1] + col_index_direction

        while coordinate_valid(row_coordinate_antinode_1, col_coordinate_antinode_1, size_df):
            coordinates_antinodes += [(row_coordinate_antinode_1, col_coordinate_antinode_1)]
            row_coordinate_antinode_1 = row_coordinate_antinode_1 + row_index_direction
            col_coordinate_antinode_1 = col_coordinate_antinode_1 - col_index_direction

        while coordinate_valid(row_coordinate_antinode_2, col_coordinate_antinode_2, size_df):
            coordinates_antinodes += [(row_coordinate_antinode_2, col_coordinate_antinode_2)]
            row_coordinate_antinode_2 = row_coordinate_antinode_2 - row_index_direction
            col_coordinate_antinode_2 = col_coordinate_antinode_2 + col_index_direction

    elif (coordinate_antenna_1[0] < coordinate_antenna_2[0]) & (coordinate_antenna_1[1] >= coordinate_antenna_2[1]):
        row_coordinate_antinode_1 = coordinate_antenna_1[0] - row_index_direction
        col_coordinate_antinode_1 = coordinate_antenna_1[1] + col_index_direction
        
        row_coordinate_antinode_2 = coordinate_antenna_2[0] + row_index_direction
        col_coordinate_antinode_2 = coordinate_antenna_2[1] - col_index_direction

        while coordinate_valid(row_coordinate_antinode_1, col_coordinate_antinode_1, size_df):
            coordinates_antinodes += [(row_coordinate_antinode_1, col_coordinate_antinode_1)]
            row_coordinate_antinode_1 = row_coordinate_antinode_1 - row_index_direction
            col_coordinate_antinode_1 = col_coordinate_antinode_1  + col_index_direction

        while coordinate_valid(row_coordinate_antinode_2, col_coordinate_antinode_2, size_df):
            coordinates_antinodes += [(row_coordinate_antinode_2, col_coordinate_antinode_2)]
            row_coordinate_antinode_2 = row_coordinate_antinode_2 + row_index_direction
            col_coordinate_antinode_2 = col_coordinate_antinode_2 - col_index_direction
        
    
    elif (coordinate_antenna_1[0] < coordinate_antenna_2[0]) & (coordinate_antenna_1[1] < coordinate_antenna_2[1]):
        row_coordinate_antinode_1 = coordinate_antenna_1[0] - row_index_direction
        col_coordinate_antinode_1 = coordinate_antenna_1[1] - col_index_direction
        
        row_coordinate_antinode_2 = coordinate_antenna_2[0] + row_index_direction
        col_coordinate_antinode_2 = coordinate_antenna_2[1] + col_index_direction

        while coordinate_valid(row_coordinate_antinode_1, col_coordinate_antinode_1, size_df):
            coordinates_antinodes += [(row_coordinate_antinode_1, col_coordinate_antinode_1)]
            row_coordinate_antinode_1 = row_coordinate_antinode_1 - row_index_direction
            col_coordinate_antinode_1 = col_coordinate_antinode_1 - col_index_direction

        while coordinate_valid(row_coordinate_antinode_2, col_coordinate_antinode_2, size_df):
            coordinates_antinodes += [(row_coordinate_antinode_2, col_coordinate_antinode_2)]
            row_coordinate_antinode_2 = row_coordinate_antinode_2 + row_index_direction
            col_coordinate_antinode_2 = col_coordinate_antinode_2 + col_index_direction
    
    
    return coordinates_antinodes

In [50]:
def count_all_antinodes_part2(data):
    df =  make_df(data)
    sorts_of_antennas = find_unique_sorts_of_antennas(data)
    sorts_of_antennas.remove('.')
    antinodes = pd.DataFrame(columns=df.columns, index=df.index)
    for antennatype in sorts_of_antennas:
        antinodes = add_new_antinodes_for_this_antennatype(df, antennatype, antinodes, part=2)
    number_of_antinodes = (antinodes == 'X').sum().sum()
    return number_of_antinodes

#test
assert count_all_antinodes_part2(test_data) == 34

In [51]:
count_all_antinodes_part2(data) 

1157